# Cardano CLI Tutorial

This interactive tutorial teaches you how to use the Cardano CLI to interact directly with a Cardano node. You'll learn to:
- Create addresses and wallets
- Build and submit transactions
- Mint your own native tokens

**Setup:** If you're using this starter kit through Demeter.run, the Cardano node and CLI are already set up for you - you're ready to go! Otherwise, ensure you have access to a Cardano node and the `cardano-cli` installed.

**How to use this notebook:** Execute each code cell one by one (click the cell and press Shift+Enter or click the play button). Some commands query the blockchain and may take a few seconds to return results.

Let's get started!

## Addresses / Wallet

### Generate keys and address

Generate a payment verification key and signing key. The verification key derives your address, while the signing key proves you own that address and allows you to spend funds from it.

In [ ]:
cardano-cli address key-gen \
    --verification-key-file my_address.vkey \
    --signing-key-file my_address.skey

Build an address from the verification key.

In [ ]:
cardano-cli address build \
    --payment-verification-key-file my_address.vkey \
    --out-file my_address.addr \
    --testnet-magic $CARDANO_NODE_MAGIC

Save the address to a variable for easy reference.

In [ ]:
MY_ADDRESS=$(cat my_address.addr)

echo "my address is: $MY_ADDRESS"

### Fund the address

All Cardano transactions require fees paid in ADA. Request test funds from the [testnet faucet](https://docs.cardano.org/cardano-testnet/tools/faucet).

**Note:** 1 ADA = 1,000,000 lovelace. The CLI displays amounts in lovelace.

### Query address funds

Verify that your address received funds from the faucet. This shows your UTxOs (Unspent Transaction Outputs) - the spendable funds at your address.

In [ ]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

## Build Transactions

Cardano transactions consume UTxOs (inputs) and create new UTxOs (outputs). Let's transfer ADA from one address to another.

### Generate recipient address

Create a second address to receive the ADA transfer.

In [ ]:
cardano-cli address key-gen \
    --verification-key-file other_address.vkey \
    --signing-key-file other_address.skey

Build the recipient address.

In [ ]:
cardano-cli address build \
    --payment-verification-key-file other_address.vkey \
    --out-file other_address.addr \
    --testnet-magic $CARDANO_NODE_MAGIC

Save the recipient address to a variable.

In [ ]:
OTHER_ADDRESS=$(cat other_address.addr)

echo "the address of the recipient is: $OTHER_ADDRESS"

### Build the transaction

Query your address to get the UTxO that will fund this transaction. A UTxO is identified by its transaction hash (TxHash) and index (TxIx).

In [ ]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

Copy the **TxHash** and **TxIx** values from the executed cell output above and paste them into the next cell, replacing the placeholder strings.

Set the transaction inputs and outputs. The `transaction build` command automatically calculates fees and balances the transaction for you.

In [ ]:
MY_TX_HASH="insert TxHash here"
MY_TX_IX="insert TxIx here"
TRANSFER_AMOUNT="50000000"

cardano-cli latest transaction build \
 --testnet-magic $CARDANO_NODE_MAGIC \
 --tx-in $MY_TX_HASH#$MY_TX_IX \
 --tx-out $OTHER_ADDRESS+$TRANSFER_AMOUNT \
 --change-address $MY_ADDRESS \
 --out-file transfer.raw

Inspect the transaction details.

In [ ]:
cardano-cli debug transaction view --tx-file transfer.raw

### Sign the transaction

Transactions must be cryptographically signed to prove you control the address spending the funds. This prevents unauthorized spending.

In [ ]:
cardano-cli latest transaction sign  \
    --signing-key-file my_address.skey  \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-body-file transfer.raw  \
    --out-file transfer.signed

jq . transfer.signed

### Submit the transaction

Submit the signed transaction to the blockchain.

In [ ]:
cardano-cli latest transaction submit \
    --tx-file transfer.signed \
    --testnet-magic $CARDANO_NODE_MAGIC

Verify the transfer by querying the recipient address.

In [ ]:
cardano-cli query utxo \
    --address $OTHER_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

## Minting Native Assets

### Create minting keys

Generate keys for your minting policy. The policy defines who can mint or burn tokens - only holders of the policy keys can mint or burn tokens under this policy.

In [ ]:
cardano-cli address key-gen \
    --verification-key-file asset_policy.vkey \
    --signing-key-file asset_policy.skey

### Hash the key

Generate the key hash needed for the policy script.

In [ ]:
POLICY_HASH=$(cardano-cli address key-hash --payment-verification-key-file asset_policy.vkey)

echo "my policy hash is $POLICY_HASH"

### Create a policy script

The policy script defines the rules for minting/burning tokens. This simple signature policy requires a signature from the policy key to mint or burn tokens.

In [ ]:
echo "{\"type\": \"sig\", \"keyHash\" : \"$POLICY_HASH\" }" > policy.script

jq . policy.script

### Compute the policy ID

The policy ID uniquely identifies your token policy.

In [ ]:
POLICY_ID=$(cardano-cli latest transaction policyid --script-file policy.script)

echo "my policy id is $POLICY_ID"

The policy ID is saved to the `POLICY_ID` variable for use in the minting transaction.

### Build the minting transaction

Query your address to get the current UTxO.

In [ ]:
cardano-cli query utxo \
    --address $MY_ADDRESS \
    --testnet-magic $CARDANO_NODE_MAGIC

Copy the **TxHash**, **TxIx**, and **Amount** values from the executed cell output above and paste them into the next cell, replacing the placeholder strings.

Set the transaction variables. You'll need the TxHash, TxIx, and Amount from the UTxO query above.

In [ ]:
TX_HASH="insert TxHash here"
TX_IX="insert TxIx here"
TX_AMOUNT="insert Amount here"

Set the transaction fee. When using `build-raw`, fees must be specified manually. We'll set an arbitrarily high fee which will easily cover the minimum transaction cost.

In [ ]:
FEE=200000

Calculate the remaining balance after fees. The transaction must be balanced - inputs must equal outputs plus fees.

In [ ]:
let REMAINING=$TX_AMOUNT-$FEE

Build the minting transaction. This transaction includes a `--mint` flag that creates new tokens and adds them to your address.

**Note:** The `TOKEN_NAME` must be hex-encoded. The value "54657374746f6b656E" is the hex encoding of "Testtoken".

In [ ]:
TOKEN_NAME="54657374746F6B656E"
TOKEN_AMOUNT="10000000"

cardano-cli latest transaction build-raw \
 --fee $FEE \
 --tx-in $TX_HASH#$TX_IX \
 --tx-out $MY_ADDRESS+$REMAINING+"$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --mint "$TOKEN_AMOUNT $POLICY_ID.$TOKEN_NAME" \
 --minting-script-file policy.script \
 --out-file mint.raw

In [ ]:
cardano-cli debug transaction view --tx-body-file mint.raw

### Sign the transaction

Minting requires two signatures: your address key (to spend the UTxO) and the policy key (to authorize minting).

In [ ]:
cardano-cli latest transaction sign  \
    --signing-key-file my_address.skey  \
    --signing-key-file asset_policy.skey  \
    --testnet-magic $CARDANO_NODE_MAGIC \
    --tx-body-file mint.raw  \
    --out-file mint.signed

### Submit the transaction

Submit the transaction to mint your tokens.

In [ ]:
cardano-cli latest transaction submit \
    --tx-file mint.signed \
    --testnet-magic $CARDANO_NODE_MAGIC

Verify your new tokens by querying your address.

In [ ]:
cardano-cli query utxo --address $MY_ADDRESS --testnet-magic $CARDANO_NODE_MAGIC

Your address should now hold the newly minted tokens.

**Reading the output:**
- **Lovelace**: Your remaining ADA balance
- **Token amount + policy.tokenName**: Your minted tokens in the format `{amount} {policyId}.{hexTokenName}`
- For example: `10000000 c7fd...eb18.54657374746f6b656e` means you have 10,000,000 units of your token

The long hex string (`c7fd...`) is your policy ID, and `54657374746f6b656e` is the hex-encoded token name ("Testtoken").